# **Tutorial for A Fully Modular Numpy-Restricted Artificial Neural Network with Adam/AdamW Optimization, Element-wise, and Input-wise Dropouts**
Author: Patrick Erickson

---



The following notebook details the Artificial Neural Network I had made from scratch, and details how to use it. We will primarily be using a breast cancer screening dataset for the majority of our model demonstrations. Note that this data is almost completely linearly seperable, and neural networks might be too complex to gain any more predictive power. This is merely for demonstrations. The model can be found on the hyperlink of the README. If wou want more information about this dataset, look at my EDA article under my logistic regression github.

### For Classification

**Step 1**: Load the following imports:

In [1]:
import sys
scripts_path = r"../"

if scripts_path not in sys.path:
    sys.path.append(scripts_path)
    
from NeuralNetworkScripts.NeuralNetwork import NeuralNetwork
from NeuralNetworkScripts.Layer import FullyConnectedLayer
from NeuralNetworkScripts.DataHandler import DataHelper
import numpy as np
import pandas as pd

**Step 2**: We will load in the dataset and do some preliminary cleaning.

In [2]:
np.random.seed(100)
df = (pd.read_csv("../datasets/breast_cancer.csv")).dropna()

columns = df.columns.to_list()
df.replace({'Class': {2: 0, 4: 1}}, inplace=True)
df = df.to_numpy()
df = df.astype(int)
split = np.split(df, [9], axis=1)
dataframe = split[0]
labels = split[1]

**Step 3:** We need to split, standardize, and one-hot encode our labels for model testing. We can do that with the following Data Helper functions:

In [3]:
labels,_ = DataHelper.toOneHot(labels)

trainSet,trainLabels,testSet,testLabels = DataHelper.trainTestSplit(dataframe,labels,testSize=.2,trainSize=.8, randomState=None)

Standardizer = DataHelper.standardizer(trainSet)
trainSet = DataHelper.standardizeCompute(trainSet,Standardizer)
testSet = DataHelper.standardizeCompute(testSet,Standardizer)

Note that if specified, the toOneHot will also give us the order of the classfication heads.

**Step 4**: Construct the model. Notice that this is the same thing as logistic regression, with the only difference that softmax splits logistic binary outcomes into 2 seperate categories.

**Therefore, this is essentially logistic regression, represented with 2 categorical variables instead of a binary variable with a decision threshold:**

In [4]:
model = NeuralNetwork(
                output = FullyConnectedLayer(numNodes=2,activation='softmax')
                )

We can look at how our model does initially:

In [5]:
loss, guesses = model.evaluate(testSet,testLabels)

predictedTrain = np.argmax(guesses, axis=1)
trueTrain = np.argmax(testLabels, axis=1)
trainAccuracy = np.mean(predictedTrain == trueTrain) * 100

print("\n Initial Loss:", loss)
print("Accuracy: {:.2f}%".format(trainAccuracy))
print(predictedTrain)
print(trueTrain)


 Initial Loss: 2.004762491639277
Accuracy: 3.65%
[1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1
 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1
 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0]


We can see that without any sort of training the model can't discern which value is which. Something interesting to note, however, is that the data is so well seperated that the model already has some very high seperation ability. If we were to switch classification heads in this example, we would get a really high accuracy. Let us save this model so we can come back to the original weights, after doing a little bit of training:

In [6]:
model.saveModel('tutorialModel')

We can reload our model as such:

In [7]:
model1 = NeuralNetwork(model='tutorialModel')

We will train our model and report our predictions as such:

In [8]:
model1.train(trainSet, trainLabels) 
loss1, guesses1 = model1.evaluate(testSet,testLabels)

predictedTest = np.argmax(guesses1, axis=1)
trueTest = np.argmax(testLabels, axis=1)
testAccuracy = np.mean(predictedTest == trueTest) * 100

print("\n Testing Loss:", loss1)
print("Accuracy: {:.2f}%".format(testAccuracy))
print("Predicted values:\n" , predictedTest)
print("True test labels:\n ", trueTrain)

loss for epoch:  100 : 0.07532821156607548
loss for epoch:  200 : 0.07914802146479588
loss for epoch:  300 : 0.07907489774325341
loss for epoch:  400 : 0.07678547022758152
loss for epoch:  500 : 0.07698665293972919
loss for epoch:  600 : 0.07900846533507364
loss for epoch:  700 : 0.07923615415545247
loss for epoch:  800 : 0.07919292662502034
loss for epoch:  900 : 0.07938860731014154
loss for epoch:  1000 : 0.07726349261228221

 Testing Loss: 0.07110450639585986
Accuracy: 97.08%
Predicted values:
 [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0]
True test labels:
  [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0

We can see that the model now correctly classifies our labels with a fairly good probability. However, notice that the model stayed at roughly the same loss the entire time. This signals that we are doing a lot more training than we need to. We can specify the hyperparameters we want to change to reduce training time with similar values. We can grab from our model of original weights:

In [9]:
model2 = NeuralNetwork(model='tutorialModel')
model2.train(trainSet, trainLabels , epochs = 10, learningRate=.01) 
loss2, guesses2 = model2.evaluate(testSet,testLabels)

predicted2 = np.argmax(guesses2, axis=1)
trueTest = np.argmax(testLabels, axis=1)
accuracy2 = np.mean(predicted2 == trueTest) * 100

print("\n Model 2's Loss:", loss2)
print("Accuracy: {:.2f}%".format(accuracy2))
print("Predicted values:\n" , predicted2)
print("True test labels:\n ", trueTest)

loss for epoch:  1 : 0.35975726099462807
loss for epoch:  2 : 0.09336012726711422
loss for epoch:  3 : 0.08976726809335504
loss for epoch:  4 : 0.08696973606715787
loss for epoch:  5 : 0.08532022603800356
loss for epoch:  6 : 0.08039855129828184
loss for epoch:  7 : 0.08023148061256852
loss for epoch:  8 : 0.08267830007813423
loss for epoch:  9 : 0.0822986724892633
loss for epoch:  10 : 0.08071978371818508

 Model 2's Loss: 0.06495904281119044
Accuracy: 97.81%
Predicted values:
 [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0]
True test labels:
  [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 

As you can see, we get roughly the same testing accuracy with only 20 epochs and even a lower learning rate (.001). But what about Using a neural network instead of simple logistic regression? Well, we can test it out by doing the following:

In [10]:
multiLayerModel = NeuralNetwork(
                hidden2 = FullyConnectedLayer(numNodes=10,activation='ReLU'),
                output = FullyConnectedLayer(numNodes=2,activation='softmax')
                )

Here we create a model with 2 hidden layers of 10 nodes each. We specify our activation as ReLU. Let's look at the initial test:

In [11]:
lossNN, guessesNN = multiLayerModel.evaluate(testSet,testLabels)

predictedNN = np.argmax(guesses, axis=1)
trueNN = np.argmax(testLabels, axis=1)
accuracyNN = np.mean(predictedNN == trueNN) * 100

print("\n Initial Loss:", loss)
print("Accuracy: {:.2f}%".format(accuracyNN))
print(predictedNN)
print(trueNN)


 Initial Loss: 2.004762491639277
Accuracy: 3.65%
[1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1
 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1
 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0]


We can see that the extra layer added more variability. Let's see if we can get a better accuracy:

In [12]:
multiLayerModel.train(trainSet, trainLabels , epochs = 100, learningRate=.01) 
loss2, guesses2 = multiLayerModel.evaluate(testSet,testLabels)

predicted2 = np.argmax(guesses2, axis=1)
trueTest = np.argmax(testLabels, axis=1)
accuracy2 = np.mean(predicted2 == trueTest) * 100

print("\n Model 2's Loss:", loss2)
print("Accuracy: {:.2f}%".format(accuracy2))
print("Predicted values:\n" , predicted2)
print("True test labels:\n ", trueTest)

loss for epoch:  10 : 0.07558268219654432
loss for epoch:  20 : 0.0675476954701537
loss for epoch:  30 : 0.062421346601433794
loss for epoch:  40 : 0.05219236788675314
loss for epoch:  50 : 0.04584868570790804
loss for epoch:  60 : 0.04292953080461816
loss for epoch:  70 : 0.039273508429475065
loss for epoch:  80 : 0.03710123256343104
loss for epoch:  90 : 0.035364450245371624
loss for epoch:  100 : 0.03500365944773811

 Model 2's Loss: 0.18660249048659144
Accuracy: 95.62%
Predicted values:
 [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0]
True test labels:
  [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0

We can play around with some regularization techniques:

In [13]:
multiLayerModel2 = NeuralNetwork(
                inputDropout=.01,
                hidden2 = FullyConnectedLayer(numNodes=10,activation='ReLU',dropout=.1),
                output = FullyConnectedLayer(numNodes=2,activation='softmax')
                )
multiLayerModel2.train(trainSet, trainLabels , epochs = 1000, learningRate=.01, loss='AdamW', weightDecay=.001) 
loss2, guesses2 = multiLayerModel2.evaluate(testSet,testLabels)

predicted2 = np.argmax(guesses2, axis=1)
trueTest = np.argmax(testLabels, axis=1)
accuracy2 = np.mean(predicted2 == trueTest) * 100

print("\n Model 2's Loss:", loss2)
print("Accuracy: {:.2f}%".format(accuracy2))
print("Predicted values:\n" , predicted2)
print("True test labels:\n ", trueTest)

loss for epoch:  100 : 0.0822857140907541
loss for epoch:  200 : 0.05240925339951676
loss for epoch:  300 : 0.09400162770290965
loss for epoch:  400 : 0.029956688150225876
loss for epoch:  500 : 0.07285851679419479
loss for epoch:  600 : 0.05607175637637846
loss for epoch:  700 : 0.0628361210765018
loss for epoch:  800 : 0.02287928553067245
loss for epoch:  900 : 0.02296147964379549
loss for epoch:  1000 : 0.026706973013689562

 Model 2's Loss: 0.24157067425462647
Accuracy: 96.35%
Predicted values:
 [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
True test labels:
  [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1

or a different activation, with more layers of differing nodes:

In [14]:
multiLayerModel3 = NeuralNetwork(
                inputDropout=.01,
                hidden1 = FullyConnectedLayer(numNodes=10,activation='tanh',dropout=.1),
                hidden2 = FullyConnectedLayer(numNodes=10,activation='tanh',dropout=.1),
                hidden3 = FullyConnectedLayer(numNodes=7,activation='tanh',dropout=.1),
                hidden4 = FullyConnectedLayer(numNodes=5,activation='tanh',dropout=.1),
                output = FullyConnectedLayer(numNodes=2,activation='softmax')
                )
multiLayerModel3.train(trainSet, trainLabels , epochs = 10090, learningRate=.0001, loss='AdamW', weightDecay=.1) 
loss2, guesses2 = multiLayerModel3.evaluate(testSet,testLabels)

predicted2 = np.argmax(guesses2, axis=1)
trueTest = np.argmax(testLabels, axis=1)
accuracy2 = np.mean(predicted2 == trueTest) * 100

print("\n Model 2's Loss:", loss2)
print("Accuracy: {:.2f}%".format(accuracy2))
print("Predicted values:\n" , predicted2)
print("True test labels:\n ", trueTest)

loss for epoch:  1009 : 0.13673353201662355
loss for epoch:  2018 : 0.07612120616803825
loss for epoch:  3027 : 0.08720683711921995
loss for epoch:  4036 : 0.08276963903071005
loss for epoch:  5045 : 0.07264054898573467
loss for epoch:  6054 : 0.11175117564031893
loss for epoch:  7063 : 0.07253831561991393
loss for epoch:  8072 : 0.06429144135399897
loss for epoch:  9081 : 0.05754815998413063
loss for epoch:  10090 : 0.08503739725848446

 Model 2's Loss: 0.05188521013937266
Accuracy: 96.35%
Predicted values:
 [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
True test labels:
  [0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0

### Regression

Let's take one of the features of our dataset, and see if we can predict its value:

In [15]:
newTrainSet = trainSet[:,:-1]
newTrainLabels = trainSet[:,-1:]
newTestSet = testSet[:,:-1]
newTestLabels = testSet[:,-1:]


Let's also use the entire set as the batch size for full gradient descent. We can specify a regression model as such:

In [16]:
multiLayerModel4 = NeuralNetwork(
                batchSize=len(newTrainSet),
                hidden1 = FullyConnectedLayer(numNodes=10,activation='sigmoid'),
                hidden2 = FullyConnectedLayer(numNodes=10,activation='sigmoid'),
                output = FullyConnectedLayer(numNodes=1,activation='mse')
                )

Let's see what our model predicts initially for the first value in our dataset:

In [17]:
guesses = multiLayerModel4.predict(newTestSet)
print("Test true Label:", newTestLabels[0])
print("Guess:",guesses[0])

Test true Label: [-0.35472957]
Guess: [-0.19764892]


We will now see what our model predicts after training:

In [18]:
multiLayerModel4.train(newTrainSet, newTrainLabels) 
guesses2 = multiLayerModel4.predict(newTestSet)
print("Test true Label:", newTestLabels[0])
print("Guess:",guesses2[0])

loss for epoch:  100 : 0.5864500200661237
loss for epoch:  200 : 0.6470681578332602
loss for epoch:  300 : 0.4296720839458292
loss for epoch:  400 : 0.381951539158609
loss for epoch:  500 : 0.4213329338733199
loss for epoch:  600 : 0.35819838090315015
loss for epoch:  700 : 0.38747620614212125
loss for epoch:  800 : 0.3701162684153294
loss for epoch:  900 : 0.3286176494412083
loss for epoch:  1000 : 0.36873791111234494
Test true Label: [-0.35472957]
Guess: [-0.29323947]


**Note:** You can use both predict and evaluate for both classification and regression. For example:

In [19]:
loss2,guesses2 = multiLayerModel4.evaluate(newTestSet,newTestLabels)
print("Total Loss: ", loss)
print("Test true Label:", newTestLabels[0])
print("Guess:",guesses2[0])

Total Loss:  2.004762491639277
Test true Label: [-0.35472957]
Guess: [-0.29323947]


Much Closer!
**Note That these are the standardized values**. If you want to see the actual vs predicted, we can do the following with the Data handler class:

In [20]:
recombinedTest = np.c_[newTestSet,newTestLabels]
recombinedGuesses = np.c_[newTestSet,guesses2]

actualTest = DataHelper.unNormalize(recombinedTest, Standardizer)
actualGuess = DataHelper.unNormalize(recombinedGuesses, Standardizer)

print("Actual true value: " , actualTest[0,-1])
print("Guess Value: " , actualGuess[0,-1])

Actual true value:  1.0
Guess Value:  1.1069904549792366


# Conclusion
---

This was a short tutorial on how to use The Neural Network. When doing classification, do not forget to take the max of a row in order to get the correct prediction. Happy training!

# Other Features

---

### Custom Trained Handwritten Digit identifier: 

Try out drawer.py and see how well the model does that was trained in this neural network architecture with your own drawn numbers!